In [ ]:
#!pip install lmdb
import chg_utils as charge_util
from ocpmodels.datasets import LmdbDataset
from ocpmodels.datasets import data_list_collater
from ase.atoms import Atoms
import numpy as np

In [ ]:
charge_util.build_charge_lmdb(
    inpath = '../shared-scratch/ethan/sample/train', 
    outpath = '../chg/100/train',
    use_tqdm=True
)

In [ ]:
charge_util.build_charge_lmdb(
    inpath = '../shared-scratch/ethan/sample/val', 
    outpath = '../chg/100/val',
    use_tqdm=True
)

In [ ]:
dataset = LmdbDataset({'src':'../chg/100/val'})

In [ ]:
data0 = dataset[0]
data1 = dataset[1]


#del(data0.charge_edges)
#del(data1.charge_edges)

#print(data0, '\n')
#print(data1)

#print(data0.edge_index)
#print(data0.charge_edges.T)
#print(data0.natoms)
print(data0)
print(len(dataset))

In [ ]:
batch = data_list_collater([dataset[0], dataset[1], dataset[2]])

In [ ]:
batch

In [ ]:
print(batch.natoms)
print(batch.charge_density)

In [ ]:
import lmdb
import pickle
from ocpmodels.preprocessing import AtomsToGraphs
import os
#import ocpmodels.charge.chg_utils as charge_util
from ocpmodels.datasets import LmdbDataset
#from ocpmodels.datasets import data_list_collater
from ase.atoms import Atoms
import numpy as np

#a2g = charge_util.AtomsToChargeGraphs()
a2g = AtomsToGraphs(max_neigh=100,
                    radius=6,
                    r_forces=False,
                    r_distances=False,
                    r_fixed=False,
                    r_energy=False)

db = lmdb.open(
        os.path.join('../chg/base_case/', 'charge.lmdb'),
        map_size=1099511627776 * 2,
        subdir=False,
        meminit=False,
        map_async=True,
    )

molecule = Atoms('HO',
                positions = [[0, 0, 1], [0, 0, 2]],
                cell = [10, 10, 8],
                pbc=True)

#molecule.charge_grid = [1, 1, 2]

#print(molecule)

data_object = a2g.convert(molecule)
data_object.charge_density = np.array([[[1,2]]])

fid = 0

print(data_object)

txn = db.begin(write=True)
txn.put(f"{fid}".encode("ascii"), pickle.dumps(data_object,protocol=-1))
txn.commit()

txn = db.begin(write=True)
txn.put(f'length'.encode('ascii'), pickle.dumps(1, protocol=-1))
txn.commit()

db.sync()
db.close()

dataset = LmdbDataset({'src':'../chg/base_case'})

print(len(dataset))
print(dataset[0])

In [ ]:
dataset = LmdbDataset({'src':'../chg/base_case'})

In [ ]:
print(len(dataset))

In [ ]:
print(dataset[0])